In [9]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()
spark= SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [10]:
spark

In [11]:
# Read all the files under the folder /public/airlines_all/airlines and create a dataframe.
# (No need to infer the schema, nor you have to define it)

In [12]:
airline_df_base =spark.read \
.format("csv") \
.load("/public/airlines_all/airlines/*")

In [13]:
airline_df_base.rdd.getNumPartitions()

960

In [14]:
! hadoop fs -ls /public/airlines_all/airlines | grep  'part-0' | wc -l

1920


1)we have 1920 part file each of 64 mb only one file was less then 60 mb.
when we created a df we got 1919 partitions.

2)This is becase - deafault block size is 128mb , In the above case 2 file can't be merged to create one partition as it will cross 128mb.

3)One file was was less then 60 - this was merged with the one 128 mb block and resulted a 128 mb block. This resulted in 1919 partitons in df

Now we increased the parition size to approx 140mb. 

2 blocks can be merged into one and when we create df now we will get 960 partitions

In [15]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()
spark= SparkSession. \
builder. \
config('spark.sql.files.maxPartitionBytes', '146800640'). \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [16]:
airline_df_modifies =spark.read \
.format("csv") \
.load("/public/airlines_all/airlines/*")

In [17]:
airline_df_modifies.rdd.getNumPartitions()

960